In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import numpy as np
import StringIO
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
np.random.seed(7)

Using Theano backend.


In [2]:
%run ../../prepare_data.py

In [156]:
def split_content(content, ratio):
    expanded_content = [el[1] if el[1] != '=' else el[0] for el in content]
    print len(content)
    unique_content = sorted(set(expanded_content))
    
    s = np.arange(len(unique_content))
    np.random.shuffle(s)
    
    split_num = math.floor(len(unique_content) * ratio)
    validate_content = []
    shuffled_unique_train_content = [unique_content[i] for i in range(len(s)) if s[i] >= split_num]

    shuffled_unique_train_content_set = set(shuffled_unique_train_content)
    shuffled_unique_validate_content = [unique_content[i] for i in range(len(s)) if s[i] < split_num]

    shuffled_unique_validate_content_set = set(shuffled_unique_validate_content)
    
    train_content = [content[i] for i in range(len(content)) if expanded_content[i] in shuffled_unique_train_content_set]
    validate_content = [content[i] for i in range(len(content)) if expanded_content[i] in shuffled_unique_validate_content_set]
    return train_content, validate_content
    
    
train_content, validate_content = split_content(content, 0.2)
print len(validate_content)
print(train_content[-1])
print(validate_content[-1])
# seen = set()
# seen_add = seen.add
# [x for x in seq if not (x in seen or seen_add(x))]

# print len(unique_content)
# print unique_content[0]
print len(different_words)
print different_words[0]
# test = np.array(content)
# print test[2]
# print len(test[:][2])
# print test.shape

536432
106535
[u'\u017evi\u017egava', u'\u017evi\u017egati', u'Vmip1d--n', u'\u017ev\xed\u017egava', u'\u017ev\xed\u017egava', u'a', u'a\r\n']
[u'\u017evi\u017egu', u'\u017evi\u017eg', u'Ncmsl', u'\u017ev\xed\u017egu', u'\u017ev\xed\u017egu', u'c', u'c\r\n']
15300
a


In [158]:
# X, y = generate_inputs()
# save_inputs('ordered_inputs.h5', X, y)
# X, y = load_inputs('ordered_inputs.h5')

# X, y = generate_matrix_inputs()
# save_inputs('ordered_matrix_inputs.h5', X, y)
# X, y = load_inputs('ordered_matrix_inputs.h5')

X, y = generate_full_matrix_inputs()
save_inputs('shuffeled_full_matrix_train_inputs.h5', X, y)
# X, y = load_inputs('../internal_representations/inputs/shuffeled_full_matrix_inputs.h5')

# X, y, X_pure = generate_full_vowel_matrix_inputs()

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!


NameError: global name 'split_content' is not defined

In [ ]:
testX, testY = generate_full_vowel_matrix_inputs()

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!

In [5]:
# X = X[:100000]
# y = y[:100000]
# def unison_shuffled_copies(a, b):
#     assert len(a) == len(b)
#     p = np.random.permutation(len(a))
#     return a[p], b[p]
# X, y = unison_shuffled_copies(X, y)
# print X.shape

# train_X = X.astype(np.float32)
# train_y = y.astype(np.int32)

num_examples = len(X) # training set size
# nn_input_dim = max_word * len(dictionary) # input layer dimensionality
# nn_output_dim = max_num_vowels * max_num_vowels # output layer dimensionality
nn_output_dim = 121
nn_hdim = 516

# Gradient descent parameters (I picked these by hand)
# epsilon = 1 # learning rate for gradient descent
# reg_lambda = 1 # regularization strength

In [6]:
nn_output_dim

121

In [7]:
# model = Sequential()
# model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Dropout(0.2))
# model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Flatten())
# model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))


np.random.seed(7)


# create model
# model = Sequential()
# model.add(Conv1D(43, (3), input_shape=(23, 43), padding='same', activation='relu'))
# model.add(Dropout(0.2))
# model.add(Conv1D(43, (3), padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(86, (3), padding='same', activation='relu'))
# model.add(Dropout(0.2))
# model.add(Conv1D(86, (3), padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(172, (3), padding='same', activation='relu'))
# model.add(Dropout(0.2))
# model.add(Conv1D(172, (3), padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# # model.add(Conv1D(43, (3), input_shape=(None, 43), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Flatten())
# model.add(Dense(1032, activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Dropout(0.2))
# model.add(Dense(516, activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Dropout(0.2))
# model.add(Dense(nn_output_dim, activation='softmax'))


# create model
model = Sequential()
model.add(Conv1D(43, (3), input_shape=(23, 43), padding='same', activation='relu'))
model.add(Dropout(0.2))
model.add(Conv1D(43, (3), padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(43, (3), input_shape=(None, 43), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
model.add(Flatten())
# model.add(Dense(1032, activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Dropout(0.2))
model.add(Dense(516, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(nn_output_dim, activation='sigmoid'))

In [8]:
epochs = 5
lrate = 0.1
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Compile model
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [9]:
# model.fit_generator((X, y), 536432/2, epochs=5)
# testX = X[:536432/16]
# print testX.shape
# testY = y[:536432/16]
# print testY.shape
model.fit(X[:429145], y[:429145], epochs=5, batch_size=10)
# model.fit(X, y, epochs=5, validation_split=0.2, batch_size=10)

Epoch 1/5
429145/429145 [==============================] - 1400s - loss: 0.0022 - acc: 0.8242  
Epoch 2/5
 62040/429145 [===>..........................] - ETA: 1252s - loss: 0.0014 - acc: 0.8963

KeyboardInterrupt: 

In [14]:
model.save('cnn_i2_s_c43-3relu_d20_c43-3relu_mp2_f_516relu_d20_121sigmoid_mse_adam_a65_10epoch.h5')

In [10]:
model = load_model('../internal_representations/models/cnn_i2_s_c43-3relu_d20_c43-3relu_mp2_f_516relu_d20_121sigmoid_mse_adam_a65_10epoch.h5')

In [6]:
h5f.close()

NameError: name 'h5f' is not defined

In [12]:
# evaluate the model
scores = model.evaluate(X[429145:], y[429145:])
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

107232/107287 [============================>.] - ETA: 0s
acc: 96.52%


In [118]:
# calculate predictions
# test = generate_input_from_word('test', max_word, dictionary)
# print test.shape
# print test.T.shape
predictions = model.predict(np.array([generate_input_from_word('biotski', max_word, dictionary)]))
# round predictions

print decode_position(predictions[0], max_num_vowels)

b_pred = 0
ind = 0
i = 0
for el in predictions[0]:
    if b_pred < el:
        b_pred = el
        ind = i
    i += 1
    
print(ind)
# rounded = [round(x[0]) for x in predictions]
# print(rounded)

[2, 0]
2


In [ ]:
predictions = model.predict(X[429145:])

In [21]:
decode_position(predictions[0], max_num_vowels)

[1, 0]

In [23]:
def test_accuracy(predictions, y):
    dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
    num_of_pred = len(predictions)
    num_of_correct_pred = 0
    for i in range(predictions.shape[0]):
        if decode_position(predictions[i], max_num_vowels) == decode_position(y[i], max_num_vowels):
            num_of_correct_pred += 1

    return (num_of_correct_pred/float(num_of_pred)) * 100


print(test_accuracy(predictions, y[429145:]))

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
107287
96.5186835311


In [17]:
predictions.shape

(107287, 121)

In [12]:
print max_num_vowels

11


In [67]:
dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
def generate_input_from_word(word, max_word, dictionary):
    x = np.zeros((max_word, len(dictionary)))
    j = 0
    for c in list(word):
        index = 0
        for d in dictionary:
            if c == d:
                x[j, index] = 1
                break
            index += 1
        j += 1
    return x

# model = load_model()
# prediction = predict(model, generate_input_from_word('hidrija'))
# print decode_position(prediction[0])

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
